In [2]:
# %pip install -qU transformers accelerate einops langchain xformers bitsandbytes
# %pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 20.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-70b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Need auth token for these
hf_auth = '<hugging face token>'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/home/hb/myenv/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 15/15 [00:35<00:00,  2.35s/it]


Model loaded on cuda:0


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/home/hb/myenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


# Stop Criteria

If we don't provide a stopping criteria the model just goes on a bit of a tangent after answering the initial question.

In [ ]:
# stop_list = ['\nUser:', '\n```\n']

# stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
# stop_token_ids

In [ ]:
# import torch

# stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
# stop_token_ids

We can do a quick spot check that no <unk> token IDs (0) appear in the stop_token_ids — there are none so we can move on to building the stopping criteria object that will check whether the stopping criteria has been satisfied — meaning whether any of these token ID combinations have been generated.

In [ ]:
# from transformers import StoppingCriteria, StoppingCriteriaList

# # define custom stopping criteria object
# class StopOnTokens(StoppingCriteria):
#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         for stop_ids in stop_token_ids:
#             if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
#                 return True
#         return False

# stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [88]:
generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task="text-generation",
    # we pass model parameters here too
    #stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=1024,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [89]:
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

## SQL local model - Chinook DB

In [119]:
from langchain import SQLDatabase, SQLDatabaseChain
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [120]:
db_chain.run("How many people are in Employee table?")



> Entering new SQLDatabaseChain chain...
How many people are in Employee table?
SQLQuery:SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
Answer:There are 8 people in the Employee table.
> Finished chain.


'There are 8 people in the Employee table.'

In [108]:
db_chain.run("How many albums by Aerosmith?")



> Entering new SQLDatabaseChain chain...
How many albums by Aerosmith?
SQLQuery:SELECT COUNT(*) FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'Aerosmith');
SQLResult: [(1,)]
Answer:There is one album by Aerosmith.
> Finished chain.


'There is one album by Aerosmith.'

In [ ]:
from langchain import SQLDatabase, SQLDatabaseChain
# db_path = "home/hb/chinook-database/ChinookDatabase/DataSources/Chinook.db"
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [110]:
db_chain.run("How many employees are there in the foobar table?")



> Entering new SQLDatabaseChain chain...
How many employees are there in the foobar table?
SQLQuery:SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
Answer:There are 8 employees in the foobar table.
> Finished chain.


'There are 8 employees in the foobar table.'

### Use Query Checker - check Cutomize Prompt

In [124]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [125]:
db_chain.run("How many albums by Aerosmith?")



> Entering new SQLDatabaseChain chain...
How many albums by Aerosmith?
SQLQuery:Please provide your answer as a rewritten query or an explanation that there are no mistakes.

OperationalError: (sqlite3.OperationalError) near "Please": syntax error
[SQL: Please provide your answer as a rewritten query or an explanation that there are no mistakes.]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Using Agent

In [65]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

toolkit = SQLDatabaseToolkit(llm=llama_model, db=db)

In [66]:
agent_executor = create_sql_agent(
    llm=llama_model,
    toolkit=toolkit,
    verbose=True
)

In [69]:
agent_executor.run("How many actors are there?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: 
Thought: Now I know which tables are relevant, I should query the schema of the most relevant tables.
Action: sql_db_schema
Action Input: actor
Observation: Error: table_names {'actor'} not found in database
Thought: Hmm, maybe I should try another table. Let me query the schema of the film table instead.
Action: sql_db_schema
Action Input: film
Observation: Error: table_names {'film'} not found in database
Thought: This is strange. It seems like there isn't a single table in this database that has anything to do with actors or films. Maybe I should try a different approach.
Action: sql_db_query_checker
Action Input: SELECT * FROM actor
Observation: 

Please provide your answer as a rewritten query (if necessary) or a statement that the original query is correct.
Thought: Now I know how to write a correct query.
Final Answer: 

> Finished chain.


''

# SQL query eval

In [95]:
import sqlite3
db_path = "/home/hb/chinook-database/ChinookDatabase/DataSources/Chinook.db"
conn = sqlite3.connect(db_path)

In [96]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Define the SQL query to retrieve table names
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
cursor.execute(sql_query)

# Fetch all table names
table_names = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for name in table_names:
    print(name[0])

Tables in the database:
Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
MediaType
Playlist
PlaylistTrack
Track


In [97]:
# Define the SQL query to select all rows from the "Employee" table
sql_query = "SELECT * FROM Employee;"

# Execute the query
cursor.execute(sql_query)

# Fetch all rows from the result
rows = cursor.fetchall()

# Print the contents of the "Employee" table
print("Contents of the Employee table:")
for row in rows:
    print(row)

Contents of the Employee table:
(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00

In [98]:
# Define the SQL query to count the number of rows in the "Employee" table
sql_query = "SELECT COUNT(*) FROM Employee;"

# Execute the query
cursor.execute(sql_query)

# Fetch the result (a single row with the count)
count_result = cursor.fetchone()

# Extract the count from the result
num_employees = count_result[0]

# Print the number of employees
print("Number of employees:", num_employees)

# Close the cursor and connection
cursor.close()
conn.close()

Number of employees: 8


# Text Gen

In [59]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [60]:
llm(prompt="Explain to me the difference between AI and ML.")

"\n\nAnswer: Sure! Artificial intelligence (AI) and machine learning (ML) are related terms, but they refer to different concepts within the field of computer science. Here's a brief explanation of each term and how they differ:\n\nArtificial Intelligence (AI):\n\n* AI refers to the broader field of research and development aimed at creating machines that can perform tasks that typically require human intelligence, such as understanding language, recognizing images, making decisions, and solving problems.\n* AI involves a wide range of techniques, including rule-based systems, decision trees, and logic-based systems.\n* AI can be applied to various industries, such as healthcare, finance, transportation, and education.\n\nMachine Learning (ML):\n\n* ML is a subset of AI that focuses specifically on developing algorithms and statistical models that enable machines to learn from data, without being explicitly programmed.\n* In other words, ML is a type of AI that allows machines to impro

In [128]:
llm(prompt="Provide SQL query to show table content")

'\n\nI am trying to provide a SQL query that will display the contents of a table. The table name is "employees" and it has columns for "employee_id", "first_name", "last_name", "department", and "salary". I want to display all the columns and their values for each row in the table.\n\nHere\'s what I have tried so far:\n```\nSELECT * FROM employees;\n```\nThis query returns an error saying that it is not a valid SQL query. I\'m not sure what I\'m doing wrong. Can someone please help me with this?\n\nAnswer: Your query is almost correct! You just need to add a space between `SELECT` and `*` to make it a valid SQL query. Here\'s the corrected version:\n```\nSELECT * FROM employees;\n```\nThis query will retrieve all rows from the "employees" table and display all columns and their values for each row.\n\nAlternatively, you can also use the `SELECT` clause to specify which columns you want to retrieve, like this:\n```\nSELECT employee_id, first_name, last_name, department, salary FROM emp

In [132]:
llm(prompt="Summarize SQL table content")

/home/hb/myenv/lib/python3.8/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


' in a simple and concise way.\n *\n * @param {object} obj - The SQL table data as an object with column names as keys and values as values.\n * @return {string} - A string representation of the SQL table data.\n */\nfunction summarizeSqlTable(obj) {\n  const columns = Object.keys(obj);\n  const rows = [];\n\n  for (const [index, value] of Object.entries(obj)) {\n    const row = {};\n    for (const column of columns) {\n      row[column] = value;\n    }\n    rows.push(row);\n  }\n\n  return rows;\n}\n\n// Example usage:\nconst tableData = {\n  "id": 1,\n  "name": "John Doe",\n  "age": 32,\n  "email": "johndoe@example.com"\n};\n\nconst summary = summarizeSqlTable(tableData);\nconsole.log(summary);\n// Output:\n// [\n//   {\n//     "id": 1,\n//     "name": "John Doe",\n//     "age": 32,\n//     "email": "johndoe@example.com"\n//   }\n// ]\n```\nThis function takes an object with column names as keys and values as values, and returns an array of objects, where each object represents a row

In [133]:
llm(prompt="Provide contents of the Employee tablet")

'.\n\t * \n\t */\n\tpublic void getEmployeeTable() {\n\t\ttry {\n\t\t\tString query = "SELECT * FROM employees";\n\t\t\tStatement stmt = con.createStatement();\n\t\t\tResultSet rs = stmt.executeQuery(query);\n\t\t\t\n\t\t\twhile (rs.next()) {\n\t\t\t\tint empId = rs.getInt("emp_id");\n\t\t\t\tString empName = rs.getString("emp_name");\n\t\t\t\tString empDept = rs.getString("department");\n\t\t\t\tdouble empSalary = rs.getDouble("salary");\n\t\t\t\t\n\t\t\t\tSystem.out.println("Employee ID: " + empId);\n\t\t\t\tSystem.out.println("Employee Name: " + empName);\n\t\t\t\tSystem.out.println("Department: " + empDept);\n\t\t\t\tSystem.out.println("Salary: " + empSalary);\n\t\t\t}\n\t\t} catch (SQLException e) {\n\t\t\te.printStackTrace();\n\t\t}\n\t}\n\t\n\t// Method to Provide contents of the Department table.\n\t * \n\t */\n\tpublic void getDepartmentTable() {\n\t\ttry {\n\t\t\tString query = "SELECT * FROM department";\n\t\t\tStatement stmt = con.createStatement();\n\t\t\tResultSet rs = st

# Save the Model

In [131]:
model.save_pretrained(".")

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported